In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import math
import pandas as pd

In [2]:
def get_soup(url):
    
    main_html = requests.get(url)
    
    soup = bs(main_html.content, 'html.parser')
    
    return soup

In [3]:
def compute_pages(soup):
    
    try:
        items = int(re.search('\d+\s+of\s+(\d+)',soup.find_all(class_='listing-results-utils-count')[0].text).groups()[0])

        pages = math.ceil(items/40)
    
    except:
        
        return None
    
    return pages

In [4]:
def get_page_postcodes(url, page):
    
    url = url + '&pn={page}'.format(page=page)
    
    soup = get_soup(url)
    
    postcodes = [postcodes.get_text().strip(' ,') for postcodes in soup.find_all(class_='sold-prices-street-postcode')]

    return postcodes   

In [5]:
def get_inner_postcodes():
    
    base_url = 'https://www.zoopla.co.uk/house-prices/browse/london/nw6/kilburn-queens-park-south-and-west-hampstead-brondesbury-park/?q=london%20nw6'
        
    n_pages = compute_pages(get_soup(base_url))
            
    postcodes = []
    
    for page in range(n_pages+1):
        
        postcodes += get_page_postcodes(url=base_url, page=page)
    
    return set(postcodes)

In [6]:
def extract_properties_from_postcode(postcode):
    
    postcode = postcode.replace(' ', '-')
    
    base_url = 'https://www.zoopla.co.uk/house-prices/london/'
    url = 'https://www.zoopla.co.uk/house-prices/london/{pc}/?pn={page}'.format(pc=postcode, page=1)
    soup = get_soup(url)
    
    n_pages = compute_pages(soup)
    
    if not n_pages:
        return None
    
    results = []
    
    for page in range(1, n_pages+1):
            
        url = 'https://www.zoopla.co.uk/house-prices/london/{pc}/?pn={page}'.format(pc=postcode, page=page)
        soup = get_soup(url)
        propierties = soup.find_all('tr', class_=re.compile('row-(even|odd)'))
            
        for i, row in enumerate(propierties[1:]):

            try:
                
                result = {x:y for x,y in zip(('address', 'date', 'price', 'attributes'), extract_sales_by_row(row))}
                result['postcode'] = postcode    
                
                results.append(result)

            except:
                
                return row
        
    
    return results

In [7]:
def extract_old_prices(panels):
    
    data = panels[0].text.split('\n\n\n')
    
    prices = [data[date].strip() for date in range(0, len(data),2)]
    dates = [data[price].strip() for price in range(1, len(data),2)]
    
    return prices, dates

In [8]:
def extract_sales_by_row(row):

    house = row.findChild(class_='browse-cell-address')
    sales = row.findChild(class_='browse-cell-date')
    attributes = row.findChild(class_='attributes').get_text().strip()

    address = house.findChild(class_='sold-prices-results-address')
    date = sales.findChild(class_='sold-prices-data')
    price = sales.findChild(class_='sold-prices-data-price')
    old_price = sales.findChildren(class_='previous-sales-panel')
    
    if address:
        address = address.get_text().strip().split('\n')[0]
    else:
        address = None
        
    if date:
        date = [date.get_text().strip()]
    else:
        date = None
        
    if price:
        price = [price.get_text().strip()]
    else:
        price = None
    
    if old_price:
        dates, prices = extract_old_prices(old_price)
        prices = price + prices
        dates = date + dates
    else:
        prices = price
        dates = date
        
        
    return address, dates, prices, attributes

In [9]:
pc = get_inner_postcodes()

In [10]:
total = []

In [12]:
for i, postcode in enumerate(pc):
    print(i, postcode)
    total.append(extract_properties_from_postcode(postcode))

0 NW6 4DN
1 NW6 5PD
2 NW6 2HB
3 NW6 7YF
4 NW6 7BA
5 NW6 6SP
6 NW6 1TT
7 NW6 6EN
8 NW6 3LW
9 NW6 5BX
10 NW6 5HE
11 NW6 1QT
12 NW6 4RX
13 NW6 7RN
14 NW6 3JH
15 NW6 6PS
16 NW6 7DN
17 NW6 5LS
18 NW6 7DZ
19 NW6 1NF
20 NW6 6BA
21 NW6 6TD
22 NW6 2BH
23 NW6 7AL
24 NW6 4TA
25 NW6 1XT
26 NW6 3ET
27 NW6 4RH
28 NW6 3QA
29 NW6 4BR
30 NW6 3RP
31 NW6 3RX
32 NW6 2DX
33 NW6 7TY
34 NW6 5JJ
35 NW6 6DG
36 NW6 2PS
37 NW6 1DU
38 NW6 7ER
39 NW6 4QZ
40 NW6 6LR
41 NW6 7SE
42 NW6 1XF
43 NW6 3QR
44 NW6 2DY
45 NW6 7BU
46 NW6 7LG
47 NW6 5HL
48 NW6 4HY
49 NW6 6BJ
50 NW6 6JH
51 NW6 2PX
52 NW6 7JN
53 NW6 4DF
54 NW6 7XJ
55 NW6 7RD
56 NW6 5LF
57 NW6 6PY
58 NW6 6QH
59 NW6 4DL
60 NW6 6SG
61 NW6 5FT
62 NW6 5EG
63 NW6 5UG
64 NW6 3NH
65 NW6 4SG
66 NW6 3PH
67 NW6 7DL
68 NW6 6AU
69 NW6 1UA
70 NW6 1QW
71 NW6 5HU
72 NW6 4DT
73 NW6 1HF
74 NW6 5TR
75 NW6 6PD
76 NW6 5AT
77 NW6 1LR
78 NW6 4QL
79 NW6 7QJ
80 NW6 7EA
81 NW6 1PS
82 NW6 7XU
83 NW6 2PY
84 NW6 4EQ
85 NW6 6EG
86 NW6 5BW
87 NW6 7YE
88 NW6 5UZ
89 NW6 5FH
90 NW6 3NU
91 NW6 4T

692 NW6 5RE
693 NW6 4RS
694 NW6 1EE
695 NW6 4AU
696 NW6 3AH
697 NW6 7PB
698 NW6 6SY
699 NW6 4DJ
700 NW6 4PG
701 NW6 6RN
702 NW6 7NR
703 NW6 1LN
704 NW6 7DR
705 NW6 3AL
706 NW6 3JL
707 NW6 6BF
708 NW6 6DE
709 NW6 5JE
710 NW6 1QG
711 NW6 4JH
712 NW6 5JP
713 NW6 6NL
714 NW6 3LB
715 NW6 3QN
716 NW6 3JN
717 NW6 2BT
718 NW6 6LA
719 NW6 5BT
720 NW6 1PN
721 NW6 3QG
722 NW6 1QR
723 NW6 5ER
724 NW6 7GZ
725 NW6 2LS
726 NW6 6DX
727 NW6 6JL
728 NW6 1PU
729 NW6 1HT
730 NW6 3BP
731 NW6 7AT
732 NW6 3EE
733 NW6 1TN
734 NW6 3HS
735 NW6 1QE
736 NW6 1PR
737 NW6 3LS
738 NW6 2PD
739 NW6 4NE
740 NW6 6PP
741 NW6 1UE
742 NW6 7PD
743 NW6 7HH
744 NW6 3HB
745 NW6 2AF
746 NW6 2BE
747 NW6 7SD
748 NW6 3RG
749 NW6 1TH
750 NW6 1DJ
751 NW6 2AP
752 NW6 5QS
753 NW6 1RL
754 NW6 4SN
755 NW6 5QT
756 NW6 6JE
757 NW6 7RJ
758 NW6 5JB
759 NW6 1HS
760 NW6 3AP
761 NW6 6BB
762 NW6 7PY
763 NW6 1TJ
764 NW6 7FB
765 NW6 7PJ
766 NW6 3RQ
767 NW6 5LB
768 NW6 7BY
769 NW6 6AA
770 NW6 4QU
771 NW6 4PT
772 NW6 1EH
773 NW6 6AE
774 NW6 7XR
775 

In [110]:
df = pd.concat([pd.DataFrame(x) for x in total])
df = df.dropna()

In [111]:
df.head()

,address,date,price,attributes,postcode
1,"Flat 3, Snowman House, Abbey Road","[Nov 2016, Feb 2008]","[£327,500, £228,950]","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN
5,"Flat 7, Snowman House, Abbey Road",[Sep 2013],"[£260,000]","Flat, Leasehold",NW6-4DN
7,"Flat 9, Snowman House, Abbey Road",[Dec 2007],"[£229,950]","Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-4DN
13,"Flat 15, Snowman House, Abbey Road",[May 2003],"[£51,000]","Flat, Leasehold",NW6-4DN
31,"Flat 33, Snowman House, Abbey Road",[Mar 2003],"[£56,500]","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN


---------

In [112]:
def extract_beds(x):
    txt = x['attributes']
    match = re.search('(\d+)\s*Beds?', txt, re.IGNORECASE)
    if match:
        return match.groups()[0]
    else:
        return None

In [113]:
def merge_price_date(x):
    price_date = []
    for price, date in zip(x['price'], x['date']):
        price_date.append(price+'_'+date)
    return price_date

In [114]:
df['beds'] = df.apply(extract_beds, axis=1)
df = df.dropna()
df['price_date'] = df.apply(merge_price_date, axis = 1) # This is done so it is easier to "melt" the df with the columns at the same time
df = df.drop(['price', 'date'], axis = 1)
df = df.reset_index(drop=True)

In [115]:
df

,address,attributes,postcode,beds,price_date
0,"Flat 3, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"[£327,500_Nov 2016, £228,950_Feb 2008]"
1,"Flat 9, Snowman House, Abbey Road","Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-4DN,1,"[£229,950_Dec 2007]"
2,"Flat 33, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"[£56,500_Mar 2003]"
3,"Flat 35, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"[£175,000_Oct 2012]"
4,"Flat 36, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"[£235,000_Aug 2013]"
...,...,...,...,...,...
8627,85 Brassey Road,"Flat, Leasehold, 0 Beds, 1 Bath, 1 Recep",NW6-2BB,0,"[£175,000_Oct 2015, £172,000_Sep 2014, £97,594..."
8628,91 Brassey Road,"Flat, Leasehold, 0 Beds, 1 Bath, 1 Recep",NW6-2BB,0,"[£190,000_Jan 2013]"
8629,104 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath",NW6-2BB,1,"[£169,000_Mar 2006, £106,000_Feb 2005]"
8630,113 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"[£372,500_Dec 2018, £210,000_Oct 2011]"


In [116]:
df = df.explode('price_date')

In [117]:
df[['price','date']] = df['price_date'].str.split('_', expand=True) 

In [118]:
df = df.drop('price_date', axis=1)

In [119]:
df

,address,attributes,postcode,beds,price,date
0,"Flat 3, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£327,500",Nov 2016
0,"Flat 3, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£228,950",Feb 2008
1,"Flat 9, Snowman House, Abbey Road","Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-4DN,1,"£229,950",Dec 2007
2,"Flat 33, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£56,500",Mar 2003
3,"Flat 35, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£175,000",Oct 2012
...,...,...,...,...,...,...
8629,104 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath",NW6-2BB,1,"£106,000",Feb 2005
8630,113 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£372,500",Dec 2018
8630,113 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£210,000",Oct 2011
8631,116 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£260,000",Oct 2012


In [120]:
df.to_csv('nw6_data.csv')

----------

3. Segment the data by property size (number of bedrooms)
3. Build a VERY SIMPLE repeat sales index of the rents and yields (http://realestate.wharton.upenn.edu/wp-content/uploads/2017/03/724.pdf).

In [124]:
df.sort_values(by='address')

,address,attributes,postcode,beds,price,date
4423,"1 Treetop Mews, Christchurch Avenue","Terraced house, Freehold, 4 Beds, 3 Baths, 1 R...",NW6-7BL,4,"£1,110,000",Jan 2014
7733,"1, 42 Fortune Green Road","Flat, Leasehold, 3 Beds, 2 Baths, 1 Recep",NW6-1UJ,3,"£370,000",Apr 2002
2477,"1, 76 Kingsgate Road","Flat, Leasehold, 2 Beds, 2 Baths, 1 Recep",NW6-4LA,2,"£725,000",Jun 2015
2477,"1, 76 Kingsgate Road","Flat, Leasehold, 2 Beds, 2 Baths, 1 Recep",NW6-4LA,2,"£425,000",May 2009
2477,"1, 76 Kingsgate Road","Flat, Leasehold, 2 Beds, 2 Baths, 1 Recep",NW6-4LA,2,"£290,000",Mar 2004
...,...,...,...,...,...,...
6071,"Upper Maisonette, 24 Narcissus Road","Flat, Share of freehold, 2 Beds, 2 Baths, 1 Recep",NW6-1TH,2,"£599,950",Nov 2007
6071,"Upper Maisonette, 24 Narcissus Road","Flat, Share of freehold, 2 Beds, 2 Baths, 1 Recep",NW6-1TH,2,"£400,000",Oct 2006
7414,"Wavel House, Wavel Mews","Detached house, Freehold, 3 Beds, 3 Baths, 2 R...",NW6-3AB,3,"£1,225,000",Aug 2012
826,"Woodstock, 83 Priory Road","Flat, Leasehold, 3 Beds, 2 Baths, 1 Recep",NW6-3NL,3,"£165,000",Jan 1999


In [129]:
valid_propierties = df['address'].sort_values().unique()[df.groupby('address')['date'].size()>1]

In [130]:
vdf = df[df['address'].isin(valid_propierties)]

In [131]:
vdf

,address,attributes,postcode,beds,price,date
0,"Flat 3, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£327,500",Nov 2016
0,"Flat 3, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£228,950",Feb 2008
1,"Flat 9, Snowman House, Abbey Road","Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-4DN,1,"£229,950",Dec 2007
2,"Flat 33, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£56,500",Mar 2003
3,"Flat 35, Snowman House, Abbey Road","Flat, Leasehold, 2 Beds, 1 Bath, 1 Recep",NW6-4DN,2,"£175,000",Oct 2012
...,...,...,...,...,...,...
8629,104 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath",NW6-2BB,1,"£106,000",Feb 2005
8630,113 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£372,500",Dec 2018
8630,113 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£210,000",Oct 2011
8631,116 Brassey Road,"Flat, Leasehold, 1 Bed, 1 Bath, 1 Recep",NW6-2BB,1,"£260,000",Oct 2012
